## FIBSEM Database (SQLModel Version)

In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem.db.v3 import models

user = models.User(username='patrick', 
                name='Patrick Cleeve',
                email = 'patrick@openfibsem.org',
                password = 'password',
                role = 'admin')

print(user.username)




In [ ]:
from sqlmodel import Session, SQLModel, create_engine, select

from fibsem import config as cfg

def create_connection(database_path: str = cfg.DATABASE_PATH, echo: bool = False):
    # create a connection to the database
    engine = create_engine(f'sqlite:///{database_path}', echo=echo)

    return engine

def create_database(engine):
    # create the database
    SQLModel.metadata.create_all(engine)

engine = create_connection(echo=True)
create_database(engine)

# with Session(engine) as session:
    # session.add(user)
    # session.commit()

In [ ]:
with Session(engine) as session:
    statement = select(models.User)
    users = session.exec(statement)
    for user in users:
        print(user)